In [1]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

# Set inline
%matplotlib inline

In [2]:
def RCoeff(frequency, theta_i_degrees, cw, cp1, rho_ratio, rhow, alphap1):
    """
    Function to compute reflection coefficient.
    """
    # Convert attenuation to loss parameter
    deltap1 = alphap1 * cp1 * np.log(10) / (40 * np.pi * frequency)

    # Convert from degrees to radians
    theta_i = np.deg2rad(theta_i_degrees)

    # Complex sound speed
    cpp = cp1 / (1 + 1j * deltap1)

    # Ratio of sound speeds
    ap = cpp / cw

    # Impedance for water
    zw = rhow * cw / (np.abs(np.sin(theta_i)) + 1e-6)

    # Impedance of sediment
    zp = rho_ratio * cpp / np.sqrt(1 - (ap * np.cos(theta_i))**2)

    # Impedance ratio
    zwp = zp / zw

    # Reflection coefficient
    R1 = (zwp - 1) / (zwp + 1)

    return R1

In [3]:
def plot_reflection(cw, cp1, rhop1, alphap1):
    """
    # Function to plot magnitude and phase of the reflection coefficient and show critical / intromission angles.
    """
    # Frequency (Hz)
    frequency = 400000

    # Water density (g/cm^3)
    rhow = 1.0

    # Grazing angles for reflection coefficient
    theta = np.arange(0, 90.25, 0.25)

    # Function to generate reflection coefficient
    R1 = RCoeff(frequency, theta, cw, cp1, rhop1/rhow, rhow, alphap1)
    
    # Determine critical angle or intromission angle

    if cp1 > cw:
        # Critical angle existence condition
        disp_angle = np.arccos(cw/cp1) * 180/np.pi
        lineColor = "red"
        annoteText = "Critical Angle"
    else:
        # If cp1 < cw, an angle of intromission exists
        disp_angle = np.arccos(np.sqrt(((rhop1/rhow)**2 - (cw/cp1)**2)/((rhop1/rhow)**2 - 1))) * 180/np.pi
        lineColor = "black"
        annoteText = "Intromission Angle"
    
    # Create a figure with two subplots
    _, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,5))
    
    # Plot the magnitude of the reflection coefficient
    ax1.plot(theta, np.abs(R1), linewidth=2)
    ax1.set_xlabel("Grazing Angle (degrees)")
    ax1.set_ylabel("Magnitude of Reflection Coefficient")

    # Draw a vertical line at the critical/intromission angle
    ax1.axvline(x=disp_angle, color=lineColor, linewidth=2)
    ax1.text(disp_angle+2, 0.95 * ax1.get_ylim()[1], annoteText, color=lineColor)
    ax1.grid(True)

    # Set y-axis limit
    ax1.set_ylim(0, 1.1 * np.max(np.abs(R1)))
    
    # Plot the phase of the reflection coefficient
    ax2.plot(theta, np.angle(R1), linewidth=2)
    ax2.set_xlabel("Grazing Angle (degrees)")
    ax2.set_ylabel("Phase of Reflection Coefficient")

    # Draw the same vertical line at the critical/intromission angle
    ax2.axvline(x=disp_angle, color=lineColor, linewidth=2)
    ax2.text(disp_angle+2, 0.95 * ax2.get_ylim()[1], annoteText, color=lineColor)
    ax2.grid(True)

    # Display the full range of phase values
    ax2.set_ylim(-np.pi, np.pi)
    
    # Show plots
    plt.tight_layout()
    plt.show()

# Create interactive widgets
layout = widgets.Layout(width="500px", height="30px")
widgets.interact(
    plot_reflection,
    cw=widgets.FloatSlider(
        min=1300,
        max=1700,
        step=1,
        value=1500, 
        description="Water Speed (m/s)", 
        style={"description_width": "initial"}, 
        layout=layout,
    ),
    cp1=widgets.FloatSlider(
        min=1300,
        max=1700,
        step=1,
        value=1500,
        description="Sediment Speed (m/s)", 
        style={"description_width": "initial"}, 
        layout=layout,
    ),
    rhop1=widgets.FloatSlider(
        min=1.2,
        max=2.5,
        step=0.1,
        value=1.6, 
        description="Sediment Density (g/cm^3)", 
        style={"description_width": "initial"}, 
        layout=layout,
    ),
    alphap1=widgets.FloatSlider(
        min=20,
        max=650,
        step=10,
        value=20, 
        description="Attenuation (dB/m)", 
        style={"description_width": "initial"}, 
        layout=layout,
    )
)


interactive(children=(FloatSlider(value=1500.0, description='Water Speed (m/s)', layout=Layout(height='30px', …

<function __main__.plot_reflection(cw, cp1, rhop1, alphap1)>